In [6]:
import os
from pdf2image import convert_from_path
import pdfplumber
import numpy as np
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage


In [18]:
PDF_DIR = '../dataset/nai_duniya/'
pdfs = os.listdir(PDF_DIR)

In [19]:
img_no = 1
for pdf in pdfs:
    img = convert_from_path(PDF_DIR + pdf)
    img = img.pop()
    img = np.asarray(img)
    
    """
    pdf = pdfplumber.open(PDF_DIR + pdf)
    first_page = pdf.pages[0]
    words = first_page.extract_words()
    im = first_page.to_image(200)
    
    nwords = []
    bbox = []
    for word in words:
        xmin = word['x0']
        xmax = word['x1']
        ymin = word['top']
        ymax = word['bottom']

        if not ((xmax-xmin) > 130 and (ymax-ymin) < 50): 
            xmax *= im.scale 
            xmin *= im.scale 
            ymax *= im.scale 
            ymin *= im.scale 

            bbox.append([xmin, ymin, xmax, ymax])
    """        
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite('../dataset/nai_duniya_png/' + str(img_no).zfill(2) + '.png', img)
    #np.save('../dataset/words_localized/' + str(img_no).zfill(2) + '.npy', bbox)
    
    img_no += 1

In [136]:
imgs = os.listdir('../dataset/words_localized/')

In [137]:
for imgn in imgs:
    if 'png' in imgn:
        img = cv2.imread('../dataset/words_localized/' + imgn)
        npy = np.load('../dataset/words_localized/' + imgn[:-4] + '.npy')
        
        for bb in npy:
            cv2.rectangle(img, (bb[0], bb[1]), (bb[2], bb[3]), (0,255,0), 1)
            
        cv2.imwrite('../dataset/words_localized/' + 'bbox_' + imgn, img)

In [6]:
imgs = os.listdir('../dataset/words_localized/')

In [7]:
nimgs = []
for img in imgs:
    if 'png' in img and 'bbox' not in img:
        nimgs.append(img)

In [112]:
img = cv2.imread('../dataset/words_localized/' + nimgs[0], 0)

In [8]:
img = cv2.imread('test1.jpg', 0)

In [9]:
from skimage import filters, segmentation


In [10]:
val = filters.threshold_otsu(img)
mask = img < val
clean_border = segmentation.clear_border(mask)

In [11]:
cv2.imwrite('clean_border.png', (1 - clean_border) * 255)


True

In [12]:
from PIL import Image
from tesserocr import PyTessBaseAPI, RIL


In [13]:
image = Image.open('clean_border.png')
with PyTessBaseAPI() as api:
    api.SetImage(image)
    boxes = api.GetComponentImages(RIL.WORD, True)
    print 'Found {} textline image components.'.format(len(boxes))

Found 480 textline image components.


In [14]:
img = cv2.imread('clean_border.png')
for bb in boxes:
    cv2.rectangle(img, (bb[1]['x'],bb[1]['y']), (bb[1]['x']+bb[1]['w'],bb[1]['y']+bb[1]['h']), (0,255,0), 1)
cv2.imwrite('bb_detected_pdf.png',img)

True

In [3]:
api.GetComponentImages?

Docstring:
Get the given level kind of components (block, textline, word etc.) as a
list of image, box bounds {x, y, width, height} tuples in reading order.

Can be called before or after :meth:`Recognize`.

Args:
    level (int): Iterator level. See :class:`RIL`.
    text_only (bool): If ``True``, then only text components are returned.

Kwargs:
    raw_image (bool): If ``True``, then portions of the original image are extracted
        instead of the thresholded image and padded with `raw_padding`. Defaults to
        ``False``.
    raw_padding (int): Image padding pixels. Defaults to 0.
    blockids (bool): If ``True``, the block-id of each component is also included
        in the returned tuples (`None` otherwise). Defaults to ``True``.
    paraids (bool): If ``True``, the paragraph-id of each component with its block
        is also included in the returned tuples.

Returns:
    list: List of tuples containing the following values respectively::

        image (:class:`PIL.Image`